In [1]:
# decision_tree.py
# ---------
# Licensing Information:  You are free to use or extend these projects for
# personal and educational purposes provided that (1) you do not distribute
# or publish solutions, (2) you retain this notice, and (3) you provide clear
# attribution to UT Dallas, including a link to http://cs.utdallas.edu.
#
# This file is part of Homework for CS6375: Machine Learning.
# Gautam Kunapuli (gautam.kunapuli@utdallas.edu)
# Sriraam Natarajan (sriraam.natarajan@utdallas.edu),
# Anjum Chida (anjum.chida@utdallas.edu)
#
#
# INSTRUCTIONS:
# ------------
# 1. This file contains a skeleton for implementing the ID3 algorithm for
# Decision Trees. Insert your code into the various functions that have the
# comment "INSERT YOUR CODE HERE".
#
# 2. Do NOT modify the classes or functions that have the comment "DO NOT
# MODIFY THIS FUNCTION".
#
# 3. Do not modify the function headers for ANY of the functions.
#
# 4. You may add any other helper functions you feel you may need to print,
# visualize, test, or save the data and results. However, you MAY NOT utilize
# the package scikit-learn OR ANY OTHER machine learning package in THIS file.

import numpy as np
import os
#import graphviz


In [6]:
from collections import defaultdict

def partition(x):
    """
    Partition the column vector x into subsets indexed by its unique values (v1, ... vk)

    Returns a dictionary of the form
    { v1: indices of x == v1,
      v2: indices of x == v2,
      ...
      vk: indices of x == vk }, where [v1, ... vk] are all the unique values in the vector z.
    """

    # INSERT YOUR CODE HERE
    Dict = defaultdict(list)
    index=0
    for i in x:
        if i[0] in Dict:
            Dict[i[0]].append(index)
        else:
            Dict.setdefault(i[0],[]).append(index)
        index+=1
    
    return Dict        
    raise Exception('Function not yet implemented!')
    
# #DEBUG
# x = np.array([[1],[2],[3],[3]]) 
# partition(x)

defaultdict(list, {1: [0], 2: [1], 3: [2, 3]})

In [3]:
def entropy(y):
    """
    Compute the entropy of a vector y by considering the counts of the unique values (v1, ... vk), in z

    Returns the entropy of z: H(z) = p(z=v1) log2(p(z=v1)) + ... + p(z=vk) log2(p(z=vk))
    """

    # INSERT YOUR CODE HERE
    n = len(y)
    
    unique_freq = np.bincount(y)
    p = unique_freq[np.nonzero(unique_freq)] / n
    
    return - np.sum(p * np.log2(p))

    raise Exception('Function not yet implemented!')

#DEBUG
#x = np.array([1,2,1,2]) 
#entropy(x)

In [5]:
def conditional_entropy(x, y):
    xy = np.c_[x, y]
    
    unique_freq = np.nonzero(np.bincount(x))
    #p_v is the probability of x=v. Input x_v is all ys where x=v 
    unique_freq = np.bincount(x)
    p_v = unique_freq[np.nonzero(unique_freq)] / len(x)
    p_v_not = 1-p_v
    entropies=[]
    entropies_rest=[]
    #H_yx=np.sum(p_v*entropy(yx_v))
    for v in set(x):
        y_new=[]
        y_rest=[]
        for element in xy:
            if element[0]==v:
                y_new.append(element[1])
            else:
                y_rest.append(element[1])
        entropies_rest.append(entropy(y_rest))        
        entropies.append(entropy(y_new))
    weighted_entropies=p_v*entropies+(p_v_not*entropies_rest)
    #print(weighted_entropies)
    return np.asarray(weighted_entropies)
    #return np.sum(p_v*entropies)

def mutual_information(x, y):
    """
    Compute the mutual information between a data column (x) and the labels (y). The data column is a single attribute
    over all the examples (n x 1). Mutual information is the difference between the entropy BEFORE the split set, and
    the weighted-average entropy of EACH possible split.

    Returns the mutual information: I(x, y) = H(y) - H(y | x)
    """

    # INSERT YOUR CODE HERE
    x = np.asarray(x)
    y = np.asarray(y)
    
    H_y = entropy(y)
    H_y_vector=np.full((len(set(x)),1), H_y)
    #print(H_y_vector)
    
    H_yx=conditional_entropy(x,y)

    # Mutual Information
    I_xy =H_y - H_yx
    
    return  I_xy

    raise Exception('Function not yet implemented!')
    
#DEBUG
x = np.array([2,3,5,3,2,2])
y = np.array([1,1,1,1,0,0])
mutual_information(x,y)

array([0.45914792, 0.25162917, 0.10917034])

In [ ]:
def create_att_val_pairs(x):
    attribute_value_pairs=[]
    col_num=0
    
    for column in x.T:
        for x in set(column):
            attribute_value_pairs.append(tuple((col_num,x)))
        col_num+=1
    return attribute_value_pairs
##DEBUG
# x = np.array([[1,0,5],[2,0,6],[3,1,5],[1,1,6],[2,0,6],[3,0,6]])
# create_att_val_pairs(x)

In [ ]:
def id3_love(x, y, attribute_value_pairs=None, depth=0, max_depth=5):
    """
    Implements the classical ID3 algorithm given training data (x), training labels (y) and an array of
    attribute-value pairs to consider. This is a recursive algorithm that depends on three termination conditions
        1. If the entire set of labels (y) is pure (all y = only 0 or only 1), then return that label
        2. If the set of attribute-value pairs is empty (there is nothing to split on), then return the most common
           value of y (majority label)
        3. If the max_depth is reached (pre-pruning bias), then return the most common value of y (majority label)
    Otherwise the algorithm selects the next best attribute-value pair using INFORMATION GAIN as the splitting criterion
    and partitions the data set based on the values of that attribute before the next recursive call to ID3.

    The tree we learn is a BINARY tree, which means that every node has only two branches. The splitting criterion has
    to be chosen from among all possible attribute-value pairs. That is, for a problem with two features/attributes x1
    (taking values a, b, c) and x2 (taking values d, e), the initial attribute value pair list is a list of all pairs of
    attributes with their corresponding values:
    [(x1, a),
     (x1, b),
     (x1, c),
     (x2, d),
     (x2, e)]
     If we select (x2, d) as the best attribute-value pair, then the new decision node becomes: [ (x2 == d)? ] and
     the attribute-value pair (x2, d) is removed from the list of attribute_value_pairs.

    The tree is stored as a nested dictionary, where each entry is of the form
                    (attribute_index, attribute_value, True/False): subtree
    * The (attribute_index, attribute_value) determines the splitting criterion of the current node. For example, (4, 2)
    indicates that we test if (x4 == 2) at the current node.
    * The subtree itself can be nested dictionary, or a single label (leaf node).
    * Leaf nodes are (majority) class labels

    Returns a decision tree represented as a nested dictionary, for example
    {(4, 1, False):
        {(0, 1, False):
            {(1, 1, False): 1,
             (1, 1, True): 0},
         (0, 1, True):
            {(1, 1, False): 0,
             (1, 1, True): 1}},
     (4, 1, True): 1}
    """
    if attribute_value_pairs == None :
        attribute_value_pairs = []
        Shape_x=x.shape[1]
        for i in range(0, Shape_x):
            for j in set(x[:, i]):
                attribute_value_pairs.append((i,j))

    if len(attribute_value_pairs) == 0 or depth == max_depth :
        count = np.bincount(np.array(y))
        val= np.argmax(count)
        return val
    
    elif all(z == y[0] for z in y) :
        return y[0]
    
    else :
        val_max = 0
        for item in attribute_value_pairs:
            tmp = []
            k = item[0]
            length=len(x)
            for i in range(0, length):
                w = x[i][k]
                if w == item[1]:
                    tmp.append(1)
                else:
                    tmp.append(0)
            val = mutual_information(tmp,y)
            if val >= val_max:
                val_max = val
                split_find = item
        
        w = split_find[1]
        k = split_find[0]
        tmp = []
        for i in range(0, len(x)):
            tmp.append(x[i][k])
            
        final =partition(tmp)[w]
        
        T_X = []
        F_X = []
        T_Y = []
        F_Y = []
        
        length=len(x)
        for i in range(0, length):
            temp = np.asarray(x[i])
            if i in final:
                T_X.append(temp)
                T_Y.append(y[i])
            else:
                F_X.append(temp)
                F_Y.append(y[i])
        
        T = attribute_value_pairs.copy()
        F = attribute_value_pairs.copy()
        T.remove(split_find)
        F.remove(split_find)
        
        final_tree=dict()
        final_tree.update({(split_find[0], split_find[1], True): id3_love(T_X, T_Y, T, depth+1, max_depth)})
        final_tree.update({(split_find[0], split_find[1], False): id3_love(F_X, F_Y, F, depth+1, max_depth)})
        return final_tree

#DEBUG
x1 = [0, 1, 1, 2, 2, 2]
x2 = [0, 0, 1, 1, 1, 0]
y = np.array([0, 0, 0, 1, 1, 0]) 
X = np.array([x1, x2]).T
print(id3_love(X, y))

In [ ]:
def id3(x, y, attribute_value_pairs=None, depth=0, max_depth=5):
    """
    Implements the classical ID3 algorithm given training data (x), training labels (y) and an array of
    attribute-value pairs to consider. This is a recursive algorithm that depends on three termination conditions
        1. If the entire set of labels (y) is pure (all y = only 0 or only 1), then return that label
        2. If the set of attribute-value pairs is empty (there is nothing to split on), then return the most common
           value of y (majority label)
        3. If the max_depth is reached (pre-pruning bias), then return the most common value of y (majority label)
    Otherwise the algorithm selects the next best attribute-value pair using INFORMATION GAIN as the splitting criterion
    and partitions the data set based on the values of that attribute before the next recursive call to ID3.

    The tree we learn is a BINARY tree, which means that every node has only two branches. The splitting criterion has
    to be chosen from among all possible attribute-value pairs. That is, for a problem with two features/attributes x1
    (taking values a, b, c) and x2 (taking values d, e), the initial attribute value pair list is a list of all pairs of
    attributes with their corresponding values:
    [(x1, a),
     (x1, b),
     (x1, c),
     (x2, d),
     (x2, e)]
     If we select (x2, d) as the best attribute-value pair, then the new decision node becomes: [ (x2 == d)? ] and
     the attribute-value pair (x2, d) is removed from the list of attribute_value_pairs.

    The tree is stored as a nested dictionary, where each entry is of the form
                    (attribute_index, attribute_value, True/False): subtree
    * The (attribute_index, attribute_value) determines the splitting criterion of the current node. For example, (4, 2)
    indicates that we test if (x4 == 2) at the current node.
    * The subtree itself can be nested dictionary, or a single label (leaf node).
    * Leaf nodes are (majority) class labels

    Returns a decision tree represented as a nested dictionary, for example
    {(4, 1, False):
        {(0, 1, False):
            {(1, 1, False): 1,
             (1, 1, True): 0},
         (0, 1, True):
            {(1, 1, False): 0,
             (1, 1, True): 1}},
     (4, 1, True): 1}
    """

    # INSERT YOUR CODE HERE. NOTE: THIS IS A RECURSIVE FUNCTION.
    if (y.size==0) :
        return 0
    #If all set of labels(y) is pure
    print("This is y")
    print(y)
    print(np.argmax(np.bincount(y)))
    
    if np.unique(y).size==1 :
        print("1")
        return np.argmax(np.bincount(y))
    #If the set of attribute-value pairs is empty
    if depth>0 and len(attribute_value_pairs)==0:
        print("2")
        return np.argmax(np.bincount(y))
    # If the max_depth is reached
    if max_depth==depth:
        print("3")
        return np.argmax(np.bincount(y))
         #Do we need to change the tree to associate the leaf nodes to the majority class for the last two anchors?
    
    #Creating attribute_value_pairs list
    attribute_value_pairs=create_att_val_pairs(x)
    
    #Calculate the Mutual Information for each of the features
    column_num=0
    selected_attribute_for_split_index=0;
    mutual_info=np.array([])
    for column in x.T:
        res=mutual_information(column,y)
        for element in res:
            mutual_info=np.append(mutual_info,np.array([element]), axis=0)
        if(mutual_info.size==0):
            return 0
    selected_attribute_for_split_index = np.argmin(mutual_info)
    
    
    #Remove from the attribute list 
    dummy=attribute_value_pairs[selected_attribute_for_split_index]
    del attribute_value_pairs[selected_attribute_for_split_index]

    #*****THIS IS THE PART THAT NEEDS TO BE EDITED*****
    # Add to the final tree
    # We split using the selected attribute
    sets = partition(np.asarray(x[:, selected_attribute_for_split_index]).reshape(len(x),1))
    
    #Recursive call  --make this binary
    res = {}
    depth+=1
    y_subset=np.array([])
    x_subset=np.array([])
    for k, v in sets.items():
        if k==dummy[1]:
            y_1 = y.take(v, axis=0)
            x_1 = x.take(v, axis=0)
            break
    
    y_subset=np.delete(y,v)
    x_subset=np.delete(x,v,0)
    
#     res['(%d, %d, %s)' % (selected_attribute_for_split_index+1, k, "True")] = id3(np.asarray(x_1), np.asarray(y_1),attribute_value_pairs, depth, max_depth=5)
#     res['(%d, %d, %s)' % (selected_attribute_for_split_index+1, dummy[1], "False")] = id3(np.asarray(x_subset), np.asarray(y_subset),attribute_value_pairs, depth, max_depth=5)
        
#     return res
    final_tree=dict()
    final_tree.update({(selected_attribute_for_split_index+1, k, True): id3(np.asarray(x_1), np.asarray(y_1), attribute_value_pairs, depth, max_depth)})
    final_tree.update({(selected_attribute_for_split_index+1, dummy[1], False): id3(np.asarray(x_subset), np.asarray(y_subset),attribute_value_pairs, depth, max_depth)})
    return final_tree
    raise Exception('Function not yet implemented!')
    
#DEBUG
x1 = [0, 1, 1, 2, 2, 2]
x2 = [0, 0, 1, 1, 1, 0]
y = np.array([0, 0, 0, 1, 1, 0]) 
X = np.array([x1, x2]).T

id3(X, y)
#pretty_print(id3(X, y))

In [ ]:
def predict_example(x, tree):
    """
    Predicts the classification label for a single example x using tree by recursively descending the tree until
    a label/leaf node is reached.

    Returns the predicted label of x according to tree
    """

    # INSERT YOUR CODE HERE. NOTE: THIS IS A RECURSIVE FUNCTION.
    raise Exception('Function not yet implemented!')

In [ ]:
def compute_error(y_true, y_pred):
    """
    Computes the average error between the true labels (y_true) and the predicted labels (y_pred)

    Returns the error = (1/n) * sum(y_true != y_pred)
    """

    # INSERT YOUR CODE HERE
    return np.sum(np.absolute(np.asarray(y_true)-np.asarray(y_pred)))/len(y_true)

    raise Exception('Function not yet implemented!')

In [ ]:
def pretty_print(tree, depth=0):
    """
    Pretty prints the decision tree to the console. Use print(tree) to print the raw nested dictionary representation
    DO NOT MODIFY THIS FUNCTION!
    """
    if depth == 0:
        print('TREE')

    for index, split_criterion in enumerate(tree):
        sub_trees = tree[split_criterion]

        # Print the current node: split criterion
        print('|\t' * depth, end='')
        print('+-- [SPLIT: x{0} = {1} {2}]'.format(split_criterion[0], split_criterion[1], split_criterion[2]))

        # Print the children
        if type(sub_trees) is dict:
            pretty_print(sub_trees, depth + 1)
        else:
            print('|\t' * (depth + 1), end='')
            print('+-- [LABEL = {0}]'.format(sub_trees))

In [ ]:
def render_dot_file(dot_string, save_file, image_format='png'):
    """
    Uses GraphViz to render a dot file. The dot file can be generated using
        * sklearn.tree.export_graphviz()' for decision trees produced by scikit-learn
        * to_graphviz() (function is in this file) for decision trees produced by  your code.
    DO NOT MODIFY THIS FUNCTION!
    """
    if type(dot_string).__name__ != 'str':
        raise TypeError('visualize() requires a string representation of a decision tree.\nUse tree.export_graphviz()'
                        'for decision trees produced by scikit-learn and to_graphviz() for decision trees produced by'
                        'your code.\n')

    # Set path to your GraphViz executable here
    os.environ["PATH"] += os.pathsep + 'C:/Program Files (x86)/Graphviz2.38/bin/'
    graph = graphviz.Source(dot_string)
    graph.format = image_format
    graph.render(save_file, view=True)

In [ ]:
def to_graphviz(tree, dot_string='', uid=-1, depth=0):
    """
    Converts a tree to DOT format for use with visualize/GraphViz
    DO NOT MODIFY THIS FUNCTION!
    """

    uid += 1       # Running index of node ids across recursion
    node_id = uid  # Node id of this node

    if depth == 0:
        dot_string += 'digraph TREE {\n'

    for split_criterion in tree:
        sub_trees = tree[split_criterion]
        attribute_index = split_criterion[0]
        attribute_value = split_criterion[1]
        split_decision = split_criterion[2]

        if not split_decision:
            # Alphabetically, False comes first
            dot_string += '    node{0} [label="x{1} = {2}?"];\n'.format(node_id, attribute_index, attribute_value)

        if type(sub_trees) is dict:
            if not split_decision:
                dot_string, right_child, uid = to_graphviz(sub_trees, dot_string=dot_string, uid=uid, depth=depth + 1)
                dot_string += '    node{0} -> node{1} [label="False"];\n'.format(node_id, right_child)
            else:
                dot_string, left_child, uid = to_graphviz(sub_trees, dot_string=dot_string, uid=uid, depth=depth + 1)
                dot_string += '    node{0} -> node{1} [label="True"];\n'.format(node_id, left_child)

        else:
            uid += 1
            dot_string += '    node{0} [label="y = {1}"];\n'.format(uid, sub_trees)
            if not split_decision:
                dot_string += '    node{0} -> node{1} [label="False"];\n'.format(node_id, uid)
            else:
                dot_string += '    node{0} -> node{1} [label="True"];\n'.format(node_id, uid)

    if depth == 0:
        dot_string += '}\n'
        return dot_string
    else:
        return dot_string, node_id, uid

In [ ]:
if __name__ == '__main__':
    # Load the training data
    M = np.genfromtxt('./monks-1.train', missing_values=0, skip_header=0, delimiter=',', dtype=int)
    ytrn = M[:, 0]
    Xtrn = M[:, 1:]

    # Load the test data
    M = np.genfromtxt('./monks-1.test', missing_values=0, skip_header=0, delimiter=',', dtype=int)
    ytst = M[:, 0]
    Xtst = M[:, 1:]

    # Learn a decision tree of depth 3
    decision_tree = id3(Xtrn, ytrn, max_depth=3)

    # Pretty print it to console
    pretty_print(decision_tree)

    # Visualize the tree and save it as a PNG image
    dot_str = to_graphviz(decision_tree)
    render_dot_file(dot_str, './my_learned_tree')

    # Compute the test error
    y_pred = [predict_example(x, decision_tree) for x in Xtst]
    tst_err = compute_error(ytst, y_pred)

    print('Test Error = {0:4.2f}%.'.format(tst_err * 100))